This Jupyter Noteboox shows how perform spatial univariate pre-whitening of GLM coefficients using the mean residual sums of squares (ResMS) output by SPM after GLM estimation. 

In [13]:
# relevant imports
import numpy as np
import rsatoolbox as rsa
from scipy.linalg import sqrtm

np.random.seed(7)


In [14]:
# create a dataset object
betas = np.random.random(size=(5, 1000))

Univariate pre-whitening involves normalizing the activity estimated in each voxel by the standard deviation of its residuals.

In [15]:
# load residual mean square
Cov = np.random.random(size=(1000,))  # mock covariance matrix
Cov = np.diag(Cov)
Cov = np.dot(Cov, Cov.T)  # Ensures the matrix is positive definite
print(Cov)
betas_prewhitened = np.matmul(betas, np.linalg.inv(sqrtm(Cov)))

[[0.17323278 0.         0.         ... 0.         0.         0.        ]
 [0.         0.03240478 0.         ... 0.         0.         0.        ]
 [0.         0.         0.04275271 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.20785234 0.         0.        ]
 [0.         0.         0.         ... 0.         0.84349214 0.        ]
 [0.         0.         0.         ... 0.         0.         0.17670256]]


In [16]:
# create a dataset object
nCond = betas_prewhitened.shape[0]
nVox = betas_prewhitened.shape[1]
# now create a dataset object
des = {'session': 1, 'subj': 1}
obs_des = {'conds': np.array(['cond_%02d' % x for x in np.arange(nCond)])}
chn_des = {'voxels': np.array(['voxel_' + str(x) for x in np.arange(nVox)])}
dataset = rsa.data.Dataset(measurements=betas_prewhitened,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)

In [17]:
# calculate euclidean distance between conditions
dist = rsa.rdm.calc_rdm(dataset, method='euclidean', descriptor='conds')
print(dist.get_matrices())

[[[    0.         19372.9029699  60465.35760982 11656.86534238
   17456.97377556]
  [19372.9029699      0.         14103.29222431  6459.20833199
    8810.56894998]
  [60465.35760982 14103.29222431     0.         22045.50967413
   18823.00223702]
  [11656.86534238  6459.20833199 22045.50967413     0.
     913.96644474]
  [17456.97377556  8810.56894998 18823.00223702   913.96644474
       0.        ]]]


In [18]:
# diagonalize ResMS
dataset = rsa.data.Dataset(measurements=betas,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)
dist = rsa.rdm.calc_rdm(dataset, method='mahalanobis', descriptor='conds', noise=np.linalg.inv(Cov))
print(dist.get_matrices())

[[[    0.         19372.9029699  60465.35760982 11656.86534238
   17456.97377556]
  [19372.9029699      0.         14103.29222431  6459.20833199
    8810.56894998]
  [60465.35760982 14103.29222431     0.         22045.50967413
   18823.00223702]
  [11656.86534238  6459.20833199 22045.50967413     0.
     913.96644474]
  [17456.97377556  8810.56894998 18823.00223702   913.96644474
       0.        ]]]
